## Deploying Docker to Azure Web Apps

Based on Mathew's tutorial located [here](https://github.com/msalvaris/batch_shipyard_notebooks/blob/master/cifar_example/train_on_azure_batch_shipyard.ipynb)

Tutorial currently **not working with CNTK Docker Image** because the Docker image is too big for the Web App (examples & tutorials aren't very big):

```
couldn't open temporary file ...: No space left on device
```

Hence, I use a **Python 3.5-slim Image and build open-mpi + install CNTK from wheel** instead

Ideally **Use Python 3.5-alpine Image**

In [96]:
import os 
import urllib
from os import path
import json
import requests
import time

In [58]:
# Check that docker is working
!docker run --rm hello-world


Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://cloud.docker.com/

For more examples and ideas, visit:
 https://docs.docker.com/engine/userguide/



### Step 1. Create WebApp Code

In [ ]:
%%bash
mkdir script
mkdir script/code

In [60]:
%%writefile script/code/model.py

import base64
import urllib
import numpy as np
import cntk
import pkg_resources
from flask import Flask, json, request
from io import BytesIO
from PIL import Image, ImageOps
from cntk import load_model, combine

app = Flask(__name__)
print("Something outside of @app.route() is always loaded")

# Pre-load model
MODEL = load_model("ResNet_18.model")
print("Loaded model: ", MODEL)
# Pre-load labels
with open('synset-1k.txt', 'r') as f:
    LABELS = [l.rstrip() for l in f]
print("Loaded {0} labels".format(len(LABELS)))

@app.route("/")
def healthy_me():
    return "healthy"

@app.route('/cntk')
def cntk_ver():
    return "CNTK version: {}".format(pkg_resources.get_distribution("cntk").version)

@app.route("/api/uploader", methods=['POST'])
def api_upload_file():
    img = Image.open(BytesIO(request.files['imagefile'].read())).convert('RGB')
    img = ImageOps.fit(img, (224, 224), Image.ANTIALIAS)
    return json.dumps(run_some_deep_learning_cntk(img))

def run_some_deep_learning_cntk(rgb_pil_image):
    # Convert to BGR
    rgb_image = np.array(rgb_pil_image, dtype=np.float32)
    bgr_image = rgb_image[..., [2, 1, 0]]
    img = np.ascontiguousarray(np.rollaxis(bgr_image, 2))

    # Use last layer to make prediction
    z_out = combine([MODEL.outputs[3].owner])
    result = np.squeeze(z_out.eval({z_out.arguments[0]: [img]}))

    # Sort probabilities 
    a = np.argsort(result)[-1]
    predicted_category = " ".join(LABELS[a].split(" ")[1:])
    
    return predicted_category

if __name__ == '__main__':
    # This is just for debugging
    app.run(host='0.0.0.0', port=5005)

Overwriting script/code/model.py


In [61]:
%%writefile script/code/requirements.txt
Flask
gunicorn
pillow

Overwriting script/code/requirements.txt


In [62]:
urllib.urlretrieve('https://github.com/ilkarman/Azure-WebApp-w-CNTK/raw/master/Model/ResNet_18.model', 'script/code/ResNet_18.model')

('script/code/ResNet_18.model',
 <httplib.HTTPMessage instance at 0x7f3e39019c68>)

In [63]:
urllib.urlretrieve('https://github.com/ilkarman/Azure-WebApp-w-CNTK/raw/master/Model/synset-1k.txt', 'script/code/synset-1k.txt')

('script/code/synset-1k.txt', <httplib.HTTPMessage instance at 0x7f3e3901d248>)

### Step 2. Create Azure Registry to host container

In [ ]:
!az login -o table

In [23]:
selected_subscription = "'.....'"

In [24]:
!az account set --subscription $selected_subscription

In [6]:
docker_registry = "ikmscontainer"
docker_registry_group = "ikmscontainergorup"

In [ ]:
!az group create -n $docker_registry_group -l southcentralus -o table

In [ ]:
!az acr create -n $docker_registry -g $docker_registry_group -l southcentralus -o table

In [ ]:
!az acr update -n $docker_registry --admin-enabled true -o table

In [10]:
json_data = !az acr credential show -n $docker_registry
docker_username = json.loads(''.join(json_data))['username']
docker_password = json.loads(''.join(json_data))['password']

In [ ]:
print(docker_username)
print(docker_password)

In [12]:
json_data = !az acr show -n $docker_registry
docker_registry_server = json.loads(''.join(json_data))['loginServer']

### Step 3. Create Docker Image

In [ ]:
!mkdir script/docker

In [ ]:
# Using CNTK docker doesn't work - too big for WebApp
"""
%%writefile script/docker/dockerfile

FROM microsoft/cntk:2.0.beta15.0-cpu-python3.5
MAINTAINER Ilia Karmanov
ADD code /code
ENV PATH /root/anaconda3/envs/cntk-py35/bin:$PATH
WORKDIR /code
RUN pip install -r requirements.txt && \
    sudo rm -R /cntk/Examples && \
    sudo rm -R /cntk/Tutorials 

EXPOSE 5005
CMD ["python", "model.py"]
"""

I don't actually use gunicorn (use flask dev server) but will change later (should be trivial).

Had to use standard python image and build+install open-mpi and then install CNTK from wheel - this generates an image of size 1.2 GB otherwise it's 5.1GB (CNTK Docker Image)

Todo: Ideally use python:3.5-alpine (but that give me some issues when building numpy for cntk)

In [66]:
%%writefile script/docker/dockerfile

FROM python:3.5-slim
MAINTAINER Ilia Karmanov
ADD code /code
ENV PATH /usr/local/mpi/bin:$PATH
ENV LD_LIBRARY_PATH /usr/local/mpi/lib:$LD_LIBRARY_PATH
WORKDIR /code
RUN apt-get update \
    && apt-get install -y --no-install-recommends wget build-essential \
    && rm -rf /var/lib/apt/lists/* \
    && wget https://www.open-mpi.org/software/ompi/v1.10/downloads/openmpi-1.10.3.tar.gz \
    && tar -xzvf ./openmpi-1.10.3.tar.gz \
    && cd openmpi-1.10.3 \
    && ./configure --prefix=/usr/local/mpi \
    && make -j all \
    && make install \
    && cd .. \
    && rm -R openmpi-1.10.3 \
    && rm openmpi-1.10.3.tar.gz \
    && pip install https://cntk.ai/PythonWheel/CPU-Only/cntk-2.0.beta15.0-cp35-cp35m-linux_x86_64.whl \
    && pip install -r requirements.txt \
    && apt-get purge -y --auto-remove wget build-essential

EXPOSE 5005
CMD ["python", "model.py"]

Overwriting script/docker/dockerfile


In [67]:
container_name = docker_registry_server + "/ilkarman/smallapp"
application_path = 'script'
docker_file_location = path.join(application_path, 'docker/dockerfile')

In [68]:
!docker login $docker_registry_server -u $docker_username -p $docker_password

Login Succeeded


In [ ]:
%%bash
docker stop $(docker ps -a -q)
docker rm $(docker ps -a -q)

In [70]:
# Running from shell:
docker_build = "docker build -t {0} -f {1} {2} --no-cache".format(container_name, docker_file_location, application_path)
docker_build

'docker build -t ikmscontainer.azurecr.io/ilkarman/smallapp -f script/docker/dockerfile script --no-cache'

In [71]:
# This will take a while; potentially run from shell instead to see output (there will be a lot)
build_out = !$docker_build

Test everything is working locally before pushing

In [83]:
# 1.23GB (ResNet_18.model is ~60MB)
#!docker images   

In [73]:
# To debug
print(container_name)
# In shell (run interactive mode):
#docker run -it $container_name /bin/bash
#conda info --env
#which python
# ... etc

ikmscontainer.azurecr.io/ilkarman/smallapp


In [74]:
test_cont = !docker run -p 5005:5005 -d $container_name

In [75]:
time.sleep(5)  # Wait to load
!curl http://0.0.0.0:5005

healthy

In [76]:
!curl http://0.0.0.0:5005/cntk

CNTK version: 2.0.beta15.0

In [80]:
hippo_url = "https://i.ytimg.com/vi/96xC5JIkIpQ/maxresdefault.jpg"
fname = urllib.urlretrieve(hippo_url, "bhippo.jpg")[0]
requests.post("http://0.0.0.0:5005/api/uploader", files={'imagefile': open(fname, 'rb')}).json()

u'hippopotamus, hippo, river horse, Hippopotamus amphibius'

In [81]:
!docker kill {test_cont[0]}

4b0d2e78dffd326700f85b11baaff03c1eff9f38bc18db0a859fb6725a6f0211


### Step 4. Push Docker Image to Registry

In [84]:
!docker push $container_name

The push refers to a repository [ikmscontainer.azurecr.io/ilkarman/smallapp]

1927db57: Preparing 
62a93f79: Preparing 
e5ed5658: Preparing 
59e7ccde: Preparing 
f04943d3: Preparing 
1927db57: Pushed  968.7 MB/964.9 MB[5Alatest: digest: sha256:2d87c4cda74738872da37a49472b9ca7da7ad68de7bbd6c7459ef7916a8db326 size: 1584


Note: the size of the image, since the Web App loads the container into RAM you would need to configure it to have more RAM than the size of the image (if this is not possible then try to remove unnecessary files in your image or use another).

Note: I'm not sure how much bigger deployed container is compared to compressed image?

### Step 5. Create Azure Web App from Docker Image

Note: Currently cannot find a CLI-way of doing this

1. Go to your Azure Portal and create a new 'Web App on Linux (preview)' resource

2. Click on 'Configure container' and select 'Private registry' under 'Image source'

3. Configure the 'App Service plan/Location' so that your Web App size has enough RAM to host the container (e.g. S3)

4. Enter the details to connect to your ACR:
```
Image and optional tag: ikmscontainer.azurecr.io/ilkarman/smallapp
Server URL: http://ikmscontainer.azurecr.io
Login username:<docker_username>
Password:<docker_password>
```

6. Create your Web App

7. Go to the 'Application Settings' blade, scroll down until you see 'App settings' and add an entry (to use whichever port you setup), and click save:
```
Key:PORT
Value:5005
```

8. You should now be able to navigate to your Azure Web App address and see your project! If not - add '.scm' just before '.azurewebsites.net' in your URL e.g. https://cntkiksmall.scm.azurewebsites.net/ to access the Kudu console and go to 'Debug console' -> 'Bash', where you can access logfiles such as:
```
cd /home/LogFiles/docker
ls
```

9. The first load may take a while - this is because the docker image is downloaded to the WebApp. You can observe this by opening the first log-file in your directory:
```
cd /home/LogFiles/docker
cat docker_13_out.log
```
    The output should look like:
    ```
    7d27bd3d7fec: Verifying Checksum
    7d27bd3d7fec: Download complete
    7d27bd3d7fec: Pull complete
    44ae682c18a3: Pull complete
    824bd01a76a3: Pull complete
    68fe59875298: Pull complete
    9ca1d7ae0c4b: Pull complete
    46beba4b643f: Pull complete
    651cd581382c: Pull complete
    Digest: sha256:1efdaef9d8c208753fe36ccff197f28c719cc5f7d0bf5ff12f839f04e76c5f98
    Status: Downloaded newer image for ikmscontainer.azurecr.io/ilkarman/smallapp:latest
    ```




In [93]:
# Success
!curl http://cntkiksmall.azurewebsites.net/

healthy

In [94]:
!curl http://cntkiksmall.azurewebsites.net/cntk

CNTK version: 2.0.beta15.0

In [97]:
hippo_url = "https://i.ytimg.com/vi/96xC5JIkIpQ/maxresdefault.jpg"
fname = urllib.urlretrieve(hippo_url, "bhippo.jpg")[0]
requests.post("http://cntkiksmall.azurewebsites.net/api/uploader", files={'imagefile': open(fname, 'rb')}).content

'<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">\n<html><head>\n<title>502 Proxy Error</title>\n</head><body>\n<h1>Proxy Error</h1>\n<p>The proxy server received an invalid\r\nresponse from an upstream server.<br />\r\nThe proxy server could not handle the request <em><a href="/api/uploader">POST&nbsp;/api/uploader</a></em>.<p>\nReason: <strong>Error reading from remote server</strong></p></p>\n<hr>\n<address>Apache/2.4.18 (Ubuntu) Server at cntkiksmall Port 80</address>\n</body></html>\n'